# Circuit-centric quantum classifier
Sebastian Molina [smolinad@unal.edu.co](mailto:smolinad@unal.edu.co)

The following notebook shows the results of the variational classifier proposed in 'Circuit-centric quantum classifiers' by Schuld, et al (2018). 

In [1]:
import tensorcircuit as tc
import tensorflow as tf

from variational_classifier import VariationalClassifier

In [2]:
K = tc.set_backend("tensorflow")
tc.set_dtype("complex128")

('complex128', 'float64')

## Amplitude encoding
In order to use this circuit efficiently, as proposed in the paper, one should encode data with amplitude encoding. The implementation of amplitude encoding was taken from [here](https://tensorcircuit.readthedocs.io/en/latest/tutorials/qml_scenarios.html?highlight=mnist).

In [9]:
# NOTE: Change to the dataset used in Farhi implementation. This was done by the means of testing. 
(x_train, y_train), (x_test, y_test) = tc.templates.dataset.mnist_pair_data(
    3, 6, loader=None
)

In [10]:
batched_ae = K.vmap(tc.templates.dataset.amplitude_encoding, vectorized_argnums=0)

In [5]:
x_train = tf.image.pad_to_bounding_box(x_train, 2, 2, 32, 32)
x_test = tf.image.pad_to_bounding_box(x_test, 2, 2, 32, 32)

In [6]:
x_train_q = batched_ae(x_train, 10)
x_test_q = batched_ae(x_test, 10)

## Variational classifier

The variational classifier receives a `rangeList`, which corresponds of the range of controlled gates.

![Variational Classifier](vc.png)

In [7]:
vc = VariationalClassifier(rangeList=[1])

In [8]:
vc.fit(x_train_q, y_train, batch_size=32, epochs=3, validation_split=0.8)

Epoch 1/3
76/76 [==============================] - 36s 220ms/step - loss: 0.8461 - binary_accuracy: 0.7823 - val_loss: 0.7439 - val_binary_accuracy: 0.9617
Epoch 2/3
76/76 [==============================] - 6s 79ms/step - loss: 0.7218 - binary_accuracy: 0.9817 - val_loss: 0.7013 - val_binary_accuracy: 0.9770
Epoch 3/3
76/76 [==============================] - 6s 85ms/step - loss: 0.6961 - binary_accuracy: 0.9844 - val_loss: 0.6838 - val_binary_accuracy: 0.9817
